# Analysis on Selection of Redteaming Dataset

In [ ]:
import os
import pandas as pd
import random
import json
from dotenv import load_dotenv
from jinja2 import Template
import openai
import openpyxl

In [ ]:
%load_ext dotenv
%dotenv
dotenv_path = ('.env')

In [ ]:
load_dotenv(dotenv_path = dotenv_path, override = True)
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_VERSION")
openai.api_base = os.getenv("AZURE_ENDPOINT")
engine = os.getenv("ENGINE")

In [ ]:
# Game Theory Prompts
dictator_prompt = 'game_prompts/dictator_prompt.jinja2'
recipient_prompt = 'game_prompts/recipient_response.jinja2'
persona_prompt = 'game_prompts/dictator_personas.jinja2'
ownership_prompt = 'game_prompts/dictator_ownership.jinja2'
history_prompt = 'game_prompts/dictator_history.jinja2'

## Dictator Game 

The Dictator Game gives one player- the trustee- the ability to decide how much of a share of money they are willing to share with a recipient. This game measures fairness. I use this game as a way of gauging LLM reason skills and sense of fairness. The rules of this game is better explained in `dictator_response.jinja2`.

Output can be found in the `dictator_reponse.txt` file. 

In [ ]:
# Framing the Prompt
options = ['does not know', 'knows']
roles = ['trustee', 'recipient']
persona = ["You are a HR Executive that lives in the cty and is the main caretaker of your children",
           "You are a rancher that lives in a rural city",
           "Choose your persona and tell me what that persona is"]
recipient_information = ["The recipient really needs this money",
                         "The recipient is a millionaire"]
input = random.choice(options)

In [ ]:
def persona_definer(options: str, roles: str, persona: str):
    persona_list = []
    option = options
    role = roles
    persona = persona
    persona_list.append(option)
    persona_list.append(role)
    persona_list.append(persona)
    return persona_list

In [ ]:
dictator_response = []

In [ ]:
# Collect Prompt and Template
with open(f'{dictator_prompt}', 'r') as dictator_file:
    dictator_template = Template(dictator_file.read())

render_dictator = dictator_template.render()

response = openai.ChatCompletion.create(
    engine=engine, # Change to deployment name
    messages = [
        {"role":"assistant","content": render_dictator}
    ], 
    temperature=1,
    max_tokens=350,
    top_p=0.85,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)
dictator_response.append(response['choices'][0]['message']['content'])

In [ ]:
dictator_response

Expected sample output: ['As a trustee, I have given $50 to Player 2. I have not disclosed how much money I have been given to the recipient.']

### Dictator Game (Random Response)

This version of the dictator game examines if the LLM behaves differently if they have a turn at being the recipient and the 'trustee' randomly gives them money. The role of the LLM is randomly assigned. More is descriped in `recipient_response.jinja2`.

Output for the responses can be found in the `recipient_response.txt` file. 

In [ ]:
recipient_response = []

In [ ]:
# Collect Prompt and Template

with open(f'{recipient_prompt}', 'r') as recipient_file:
    recipient_template = Template(recipient_file.read())
render_response = recipient_template.render(
    previous_rounds = dictator_response + recipient_response,
    total = random.randint(80, 120),
    amount = random.randint(0, total)
    )

response2 = openai.ChatCompletion.create(
    engine=engine, # Change to deployment name
    messages = [
        {"role":"assistant","content": render_response}
    ], 
    temperature=1,
    max_tokens=350,
    top_p=0.85,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)

recipient_response.append(response2['choices'][0]['message']['content'])

In [ ]:
recipient_response

### Dictator Game (Low Giving Bot)

In this version of the dictator game, if the LLM is the recipient. The `recipient_response.jinja2` prompt is used here. I use to to explore if the LLM behaves differently when it receives a low amount taking its history into account. 

Output for the responses can be found in the `recipient_response.txt` file. 

In [ ]:
low_response =  []

In [ ]:
# Collect Prompt and Template
with open(f'{recipient_prompt}', 'r') as recipient_file:
    recipient_file = Template(recipient_file.read())

render_response = recipient_template.render(
    previous_rounds = dictator_response + recipient_response + low_response,
    total = random.randint(80, 120),
    amount = random.randint(0, 20)
    )

response3 = openai.ChatCompletion.create(
    engine=engine, # Change to deployment name
    messages = [
        {"role":"assistant","content": render_response}
    ], 
    temperature=0,
    max_tokens=350,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)

low_response.append(response3['choices'][0]['message']['content'])

In [ ]:
low_response

### Dictator Game (Framing)

If the LLM gets graming with information explicitly about it gave vs received, how will it behave? I explore this below using the prompt `dictator_history.jinja2`.

The output for this set of runs is in the `framed_response.txt` file. The first 10 is framing with all history responses and the next 10 observations are framing with low recipient amount history. 

In [ ]:
framing_response = []

In [ ]:
# Framing with all history

with open(f'{history_prompt}', 'r') as history_file:
    history_template = Template(history_file.read())

render_history = history_template.render(
    trustee_history = dictator_response + framing_response,
    recipient_history = low_response + recipient_response
    )

response3 = openai.ChatCompletion.create(
    engine=engine, # Change to deployment name
    messages = [
        {"role":"assistant","content": render_history}
    ], 
    temperature=1,
    max_tokens=350,
    top_p=0.85,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)

framing_response.append(response3['choices'][0]['message']['content'])

In [ ]:
framing_response

In [ ]:
low_framing = []

In [ ]:
# Framing by showing only low reponse history

render_history = history_template.render(
    trustee_history = dictator_response + framing_response,
    recipient_history = low_response
    )

response3 = openai.ChatCompletion.create(
    engine=engine, # Change to deployment name
    messages = [
        {"role":"assistant","content": render_history}
    ], 
    temperature=1,
    max_tokens=350,
    top_p=0.85,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)

low_framing.append(response3['choices'][0]['message']['content'])

In [ ]:
low_framing

### Dictator Persona 
Let's see if framing the identity of the LLM makes it behave any differently. The prompt for the personas based game is at `dictator_personas.jinja2`

**HR Executive**: The LLM has the persona of a HR executive living in the city and a caretaker of two children. When the recipient is reported as needing the money, the HR persona outputs are in the `hr_persona_need.txt` file. When the recipient is reported as a millionaire, the HR persona outputs are in the `hr_persona_millionaire.txt` file.

**Rancher**: The LLM has the persona of a rancher from a rural city. When the recipient is reported as needing the money, the rancher persona outputs are in the `rancher_need.txt` file. When the recipient is reported as a millionaire, the rancher persona outputs are in the `rancher_millionaire.txt` file.

Recipient Information

**Needs Money**: The LLM receives information that the recipient needs the money

**Millionaire**: The LLM receives information that the recipient is a millionaire.

In [ ]:
hr_persona = []
rancher_persona = []

In [ ]:
# HR persona w/ someone that Needs the money
with open(f'{persona_prompt}', 'r') as persona_file:
    persona_template = Template(persona_file.read())

render_persona = persona_template.render(
    persona = persona[0], # HR = 0, Rancher = 1
    input = random.choice(options),
    information = recipient_information[0] # Need Money = 0, Millionaire = 1
    )

response4 = openai.ChatCompletion.create(
    engine=engine, # Change to deployment name
    messages = [
        {"role":"assistant","content": render_persona}
    ], 
    temperature=1,
    max_tokens=350,
    top_p=0.85,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)

hr_persona.append(response4['choices'][0]['message']['content'])

In [ ]:
hr_persona

In [ ]:
millioanire_hr = []

In [ ]:
# HR with millionaire 

render_persona = persona_template.render(
    persona = persona[0], # HR = 0, Rancher = 1
    input = random.choice(options),
    information = recipient_information[1] # Need Money = 0, Millionaire = 1
    )

response4 = openai.ChatCompletion.create(
    engine=engine, # Change to deployment name
    messages = [
        {"role":"assistant","content": render_persona}
    ], 
    temperature=1,
    max_tokens=350,
    top_p=0.85,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)

millioanire_hr.append(response4['choices'][0]['message']['content'])

In [ ]:
millioanire_hr

In [ ]:
rancher_persona = []

In [ ]:
# Rancher w/ someone that needs money

render_persona = persona_template.render(
    persona = persona[1], # HR = 0, Rancher = 1
    input = random.choice(options),
    information = recipient_information[0] # Need Money = 0, Millionaire = 1
    )

response4 = openai.ChatCompletion.create(
    engine=engine, # Change to deployment name
    messages = [
        {"role":"assistant","content": render_persona}
    ], 
    temperature=1,
    max_tokens=350,
    top_p=0.85,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)

rancher_persona.append(response4['choices'][0]['message']['content'])

In [ ]:
rancher_persona

In [ ]:
millionaire_rancher = []

In [ ]:
# Rancher w/ millionaire

render_persona = persona_template.render(
    persona = persona[1], # HR = 0, Rancher = 1
    input = random.choice(options),
    information = recipient_information[1] # Need Money = 0, Millionaire = 1
    )

response4 = openai.ChatCompletion.create(
    engine=engine, # Change to deployment name
    messages = [
        {"role":"assistant","content": render_persona}
    ], 
    temperature=1,
    max_tokens=350,
    top_p=0.85,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)

millionaire_rancher.append(response4['choices'][0]['message']['content'])

In [ ]:
millionaire_rancher

## Dictator Game Exit Interview
Now that the LLM has played the game up until now, I ask the LLM to agree or disagree with the following:
1. The money I have been give rightfully is mine.
2. If I choose to give, this means I am being nice.
3. It is not fair to share this money.
4. The recipient has not earned this money. 
5. I am more deserving of the money than the recipient. 
6. If the person I am playing with is rich, I deserve the money more.
7. If the perosn I am playing with is poor, they deserve the money more. 

The responses from the LLM to this is found in `exit_interview.txt`

In [ ]:
# Collect Prompt and Template
with open(f'{ownership_prompt}', 'r') as ownership_file:
    ownership_template = Template(ownership_file.read())

render_ownership = ownership_template.render()

response5 = openai.ChatCompletion.create(
    engine=engine, # Change to deployment name
    messages = [
        {"role":"assistant","content": render_ownership}
    ], 
    temperature=1,
    max_tokens=350,
    top_p=0.85,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)

In [ ]:
response5['choices'][0]['message']['content']